<h1>Introduction and purpose<h1>

<p> Variational Quantum Eigensolver(VQE) has 3 main steps -
    <br>
    1. State Preparation: A parameterized quantum state ${\rvert}\psi(\overrightarrow{\theta}){\rangle}$ is prepared on the quantum
    device.
    <br>
    2. Energy Estimation: A Hamiltonian $H$ can always be expressed as a sum of local terms 
    $H = \sum_{i=1}^k c_i P_i$involving the product of Pauli operators $P_i$. This is also done on a quantum computer. 
    Recall that VQE relies on the variational principle which states that for any trial 
    wavefunction ${\rvert}\psi(\overrightarrow{\theta}){\rangle}$, the expectation value of the Hamiltonian 
    $H$ $$ {\langle}\psi(\overrightarrow{\theta}){\rvert}H{\rvert}\psi(\overrightarrow{\theta}){\rangle} \ge E_0$$   
where $E_0$ denotes the lowest eigenvalue of $H$. Hence, to find the lowest eigenstate (ground state) of $H$, 
one must find the set of parameters $\overrightarrow{\theta}$ that minimizes the 
expectation ${\langle}\psi(\overrightarrow{\theta}){\rvert}H{\rvert}\psi(\overrightarrow{\theta}){\rangle}$.
    <br>
    3. Classical Feedback: The parameters $\overrightarrow{\theta}$ of the quantum state are updated using a classical non-linear 
    optimization routine (make a better guess of the next waveform (variational form). This is the part we will discuss here.
    <br>
    4. Steps 1, 2 and 3 are repeated until convergence criteria (e.g. lowest energy or other variable) are met.[1][5]
    
        
</p>

<img src="Fig1.jpg"/> 

<h1>Types of classical optimizers<h1>

<p> In general, there are two kinds of optimizers:
<br>  
1. Deterministic
    a. Line search  GD, SQP, BGFS
    b. Trust regions NM, COBYLA
2. Stochastic
    a. SPSA 
    
    
    
    
1) Gradient based:  This approach requires many measurements and is not used in Qiskit.Gradient-based methods use search directions defined by the gradients of the objective function at certain points in real number space. For some given problems, the calculation of the required derivatives can become computationally demanding. On the other hand, gradient-based methods are generally preferred when the parameter space becomes large. Thus, in some situations, they are the method of choice. 
<br>
2) Non-gradient based:  These are used in Qiskit. Instead of measuring the gradient exactly, we instead approximate the values.For some optimization problems, information about the derivative of the objective function f is
unavailable, unreliable or impractical to obtain. For instance, f might be non-smooth, noisy or time-consuming to evaluate.
So that, in those cases, methods that rely on derivatives may be of little use. We call gradient-free methods those
optimization algorithms that do not use derivatives or finite differences. Example of these are L-BFGS-B(Limited-memory Broyden-Fletcher-Goldfarb-Shanno Bound), SLSQP(Sequential Least SQuares Programming optimizer), SPSA(Simultaneous Perturbation Stochastic Approximation), COBYLA(Constrained Optimization by Linear Approximation) etc.
<br>
Out of these we see SPSA performs the best in noisy real quantum hardware but COBYLA, SLSQP and L-BFGS-B perform the best (better than all others and SPSA) in noiseless simulation situations. Because of their better performance in optimizing VQE we will discuss these 4 below.[1][2]
    
    
  #Jade NB: 
    -Qiskit supports optimizers which compute analytic gradients such as CG and ADAM: https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/operators/02_gradients_framework.ipynb <br>
    -I still call L-BFGS-B a gradient-based method because it approximates the Hessian on the fly. True gradient-free methods are Powell and Nelder-Mead. http://scipy-lectures.org/advanced/mathematical_optimization/#gradient-based-methods  
    <br>
    -The dependence of the objective function on the parameters depend upon the ansatz being used and is in general  nonlinear and non-convex. In many cases local optima are sufficient and prior knowledge of a problem offers high quality starting points for the optimization. In quantum chemistry, nonlinear procedures such as Hartree–Fock utilize very good local optima and benefit greatly from high quality starting guesses. The use of high quality starting guesses will likely be important for all types of ansatz. 
</p>    

<h2>L-BFGS-B<h2>

<p> 
        This is the intuition behind the Limited-memory Broyden-Fletcher-Goldfarb-Shanno Bound (L-BFGS-B) which is a modification from BFGS which a quasi-Newton
    method meaning a compromise between gradient descent and Newton's method. Gradient Descent does linear convergence
    where Newton's method is able to do quadratic convergence. BFGS tries to improve the speed better 
    than gradient descent but avoids the very expensive inversion of the Hessian matrix
    needed in Newton's method. BFGS is able to bring down asymtote to O($N^{2}$) from Newton's O($N^{3}$). 
    But even that may be very large for large N. In the L-BFGS or limited memory version instead of iteratively
    calculating the $NxN$ metrix (the replacement of the inverted Hessian), it just calculates say for first hundred 
    pairs of Secant equation terms, so it becomes from O($N^{2}$) to O($200N$) which will be a significant reduction for
    large N. 
    <br>
    <br>
    The target goal of L-BFGS-B is to minimize the value of a differentiable scalar function :math:`f`.
    This optimizer is a quasi-Newton method, meaning that, in contrast to Newtons's method,
    it does not require $\nabla{f}^2$ Hessian (the matrix of $f(x)$ second derivatives)
    when attempting to compute :math:`f`'s minimum value.
    Like BFGS, L-BFGS is an iterative method for solving unconstrained, non-linear optimization
    problems, but approximates BFGS using a limited amount of computer memory.
    L-BFGS starts with an initial estimate of the optimal value, and proceeds iteratively
    to refine that estimate with a sequence of better estimates.
    The derivatives of :math:`f` are used to identify the direction of steepest descent,
    and also to form an estimate of the Hessian matrix (second derivative) of :math:`f`.
    L-BFGS-B extends L-BFGS to handle simple, per-variable bound constraints.
    Uses scipy.optimize.fmin_l_bfgs_b. For further detail, please refer to
    https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html        
    Also for further information check 'Qiskit implementation 
    <https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers>'_ and 'Qiskit Documentation
    <https://qiskit.org/documentation/apidoc/qiskit.algorithms.optimizers.html>'_
     <br>   
    Also watch the following youtube videos [14][15].    
</p>

[Click this link for more details and an example of the basic BFGS algorithm](./BFGS.ipynb)

<h2>SLSQP<h2>

<p> The intuition behind SLSQP(Sequential Least SQuares Programming optimizer) is it tries to approximate 
    the objective function through a parabolic function based on three points on the objective function
    and iteratively adjusts depending on whether it has crossed over or did not reach the global minimum yet
    to make it converge to the right minimum. 
    <br>
    It is part of the Sqequential Quadratic Programming(SQP) class of algorithms. 
    <br>
    SLSQP minimizes a function of several variables with any combination of bounds, equality
    and inequality constraints. The method wraps the SLSQP Optimization subroutine originally
    implemented by Dieter Kraft.
    SLSQP is ideal for mathematical problems for which the objective function and the constraints
    are twice continuously differentiable. Note that the wrapper handles infinite values in bounds
    by converting them into large floating values.
    Uses scipy.optimize.minimize SLSQP.
    For further detail, please refer to 
    'Scipy Doc <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html>'_
    Also for further information check 'Qiskit implementation <https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers>'_ and 'Qiskit Documentation
<https://qiskit.org/documentation/apidoc/qiskit.algorithms.optimizers.html>'_[3]
    <br>
    Also watch the following youtube video [16]
    
</p>

[Click here for an example of the basic SQP algorithm](./SQP.ipynb)

<h2>SPSA<h2>

<p>
    SPSA(Simultaneous Perturbation Stochastic Approximation) is a stochastic method which uses random noise as part of input or injected noise, there are situations
    where this kind of stochastic methods (like Monte Carlo) may be better than deterministic methods, also
    being unable to calculate the gradient this method is somewhat immune to getting locked into local minima.[12]
    <br>
    SPSA is an gradient descent method for optimizing systems with multiple unknown parameters.
    As an optimization method, it is appropriately suited to large-scale population models,
    adaptive modeling, and simulation optimization.[1]
    Many examples are presented at the `SPSA Web site <http://www.jhuapl.edu/SPSA>`__.
    The main feature of SPSA is the stochastic gradient approximation, which requires only two
    measurements of the objective function, regardless of the dimension of the optimization
    problem.
    SPSA can be used in the presence of noise, and it is therefore indicated in situations
        involving measurement uncertainty on a quantum computation when finding a minimum.
        If you are executing a variational algorithm using a Quantum ASseMbly Language (QASM)
        simulator or a real device, SPSA would be the most recommended choice among the optimizers
        provided here.
    The optimization process can includes a calibration phase if neither the ``learning_rate`` nor
    ``perturbation`` is provided, which requires additional functional evaluations.
    (Note that either both or none must be set.)
    For further information check 'Qiskit implementation <https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers>'_ and 'Qiskit Documentation
    <https://qiskit.org/documentation/apidoc/qiskit.algorithms.optimizers.html>'_
    <br>
     Also watch the following youtube video [17]   
        
</p>

<img src="Fig2.jpg"/> 

<h2>COBYLA<h2>

<p> The basic intuition behind COBYLA(Constrained Optimization by Linear Approximation) is it is built on the principle of simplex method which Nelder-Mead follows. COBYLA adds
    a layer of constraints from linear programming and fits a constrained boundary within the simplex paradigm and adjusts parameters like radius, height and arm length accordingly. Please see chapter 5.3 of [13].
<br>    
COBYLA is a numerical optimization method for constrained problems where the derivative of the objective function is not known.It operates by evaluating the objective function and the constrains at the vertices of a trust region. Given an optimization problem of N variables, the trust region is a simplex of N + 1 vertices (a polytope of N + 1 vertices in N dimensions). Then, the values of the objective function are calculated at each vertex of the simplex. A linear polynomial is used to create an interpolation of the objective function. Similarly, a linear interpolation of the constraints is created. Uses scipy.optimize.minimize COBYLA. For further detail, please refer to 'Scipy Doc
<https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html>'_
<br>    
Also for further information check 'Qiskit implementation <https://github.com/Qiskit/qiskit-terra/tree/main/qiskit/algorithms/optimizers>'_ and 'Qiskit Documentation
<https://qiskit.org/documentation/apidoc/qiskit.algorithms.optimizers.html>'_ [3][4]    
<br>    
For a better idea of Nelder Mead watch [18]
    
</p>        

<h2>Conclusion</h2>

<p>
    
It is important to note that these classical optimizers works in cohesion with 
There have been many suggestions for improving VQE, such as conditional variance at risk(CVar) [6], quantum-optimal-control-inspired ansatz for variational quantum algorithms [7], adaptive informationally complete POVMs for near-term quantum algorithms [8] as well as interesting use cases of VQE like in protein folding research [9] and the reverse chemistry problem of finding an ideal molecule given a property [10] have been proposed. All of these use cases require VQE to scale up and work more efficiently, meaning converge on a good answer given a larger number of variables and qubits.
In this regard, the idea of "warm start" by taking the initial state as the solution for the relaxed optimization problem (easier to solve classically) in the context of QAOA [11] has been tried.
There may be some promise of using certain initial states, for example starting with all zero, all hadamard, alternating zero and hadamard etc...  to get better convergence in VQE for larger problems. The results may further be improved if we feed the output of the final state (using one of these initial states) as the initial state for the next series of algorithm runs, basically boot-strapping or warm starting. 

</p>

<h2>References<h2>

[1]  Kandala et al., Hardware-efficient Variational Quantum Eigensolver for Small Molecules and
Quantum Magnets https://arxiv.org/pdf/1704.05018.pdf<br>
    
[2] Giacomo Nannicini, Performance of hybrid quantum/classical variational heuristics for combinatorial
optimization https://arxiv.org/pdf/1805.12037.pdf   

[3]  Miguel Paredes Quiñones and Catarina Junqueira2, Modeling Linear Inequality Constraints in Quadratic Binary
Optimization for Variational Quantum Eigensolver https://arxiv.org/pdf/2007.13245.pdf

[4]  Rattew et al., A Domain-agnostic, Noise-resistant, Hardware-efficient Evolutionary Variational Quantum Eigensolver  
https://arxiv.org/pdf/1910.09694.pdf   
    
[5] Keijzer et al., Optimization of the Variational Quantum Eigensolver for Quantum Chemistry Applications
https://arxiv.org/pdf/2102.01781.pdf

[6] Barkoutsos et al., Improving Variational Quantum Optimization using CVaR. https://arxiv.org/pdf/1907.04769.pdf

[7] Choquette et al., Quantum-optimal-control-inspired ansatz for variational quantum algorithms. https://arxiv.org/abs/2008.01098.pdf

[8] Garcia et al. Learning to measure: adaptive informationally complete POVMs for near-term quantum algorithms https://arxiv.org/abs/2104.00569.pdf

[9] Robert et al., Resource-Efficient Quantum Algorithm for Protein Folding https://arxiv.org/abs/1908.02163.pdf

[10] Barkoutsos et al., Quantum algorithm for alchemical optimization in material design https://arxiv.org/abs/2008.06449.pdf

[11] Egger et al., Warm-starting quantum optimization https://arxiv.org/abs/2009.10095.pdf

[12] Spall, Multivariate Stocastic approximation using a simulatenuous perturbation gradient approximation - http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.19.4562&rep=rep1&type=pdf

[13] Lea Miko Versbach - EVALUATION OF A GRADIENT FREE AND A GRADIENT BASED OPTIMIZATION
ALGORITHM FOR INDUSTRIAL BEVERAGE PASTEURISATION DESCRIBED BY DIFFERENT MODELING VARIANTS              - https://lup.lub.lu.se/luur/download?func=downloadFile&recordOId=8881844&fileOId=8881852

[14] Quasi Newton Methods - https://www.youtube.com/watch?v=UvGQRAA8Yms&list=PLXsmhnDvpjORzPelSDs0LSDrfJcqyLlZc&index=38

[15] Quasi Newton and BFGS - https://www.youtube.com/watch?v=QGFct_3HMzk&list=PLXsmhnDvpjORzPelSDs0LSDrfJcqyLlZc&index=39

[16] Applied Optimization - Sequential Quadratic Approximation - https://www.youtube.com/watch?v=rR6dUWT-7Ls&list=PL2bYAB99Iq4lv70uqQQAHtv8-oONUF35k&index=16

[17] NIPS 2012 Tutoral - James Spall - Stochastic Search and Optimization - https://www.youtube.com/watch?v=kVJQkmYU2VA

[18] Stats 102A Lesson 8-2 Nelder Mead Method / Algorithm - https://www.youtube.com/watch?v=vOYlVvT3W80

    